In [6]:
import pandas as pd

train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [7]:
#Feature Engineerng
brand_category = {
    # Economy/Standard
    'Toyota': 'economy', 'Honda': 'economy', 'Ford': 'economy',
    'Chevrolet': 'economy', 'Nissan': 'economy', 'Hyundai': 'economy',
    'Kia': 'economy', 'Mazda': 'economy', 'Subaru': 'economy',
    'Volkswagen': 'economy', 'Dodge': 'economy', 'RAM': 'economy',
    'Jeep': 'economy', 'GMC': 'economy', 'Buick': 'economy',
    'Mitsubishi': 'economy', 'Chrysler': 'economy', 'FIAT': 'economy',
    'Suzuki': 'economy', 'Scion': 'economy', 'smart': 'economy',
    'MINI': 'economy', 'Rivian': 'economy', 'Hummer': 'economy',
    'Pontiac': 'economy', 'Saturn': 'economy', 'Plymouth': 'economy',
    'Saab': 'economy', 'Mercury': 'economy', 'Polestar': 'economy',
    
    # Luxury
    'Mercedes-Benz': 'luxury', 'BMW': 'luxury', 'Audi': 'luxury',
    'Lexus': 'luxury', 'INFINITI': 'luxury', 'Acura': 'luxury',
    'Cadillac': 'luxury', 'Lincoln': 'luxury', 'Volvo': 'luxury',
    'Genesis': 'luxury', 'Land': 'luxury', 'Jaguar': 'luxury',
    'Alfa': 'luxury', 'Porsche': 'luxury', 'Tesla': 'luxury',
    
    # Ultra-luxury
    'Rolls-Royce': 'ultra_luxury', 'Bentley': 'ultra_luxury',
    'Maserati': 'ultra_luxury', 'Ferrari': 'ultra_luxury',
    'Lamborghini': 'ultra_luxury', 'McLaren': 'ultra_luxury',
    'Aston': 'ultra_luxury', 'Bugatti': 'ultra_luxury',
    'Maybach': 'ultra_luxury', 'Lotus': 'ultra_luxury',
    'Karma': 'ultra_luxury', 'Lucid': 'ultra_luxury'
}

# Apply the mapping
train_data['brand_category'] = train_data['brand'].map(brand_category)
test_data["brand_category"] = test_data["brand"].map(brand_category)

In [29]:
#Impute
train_data["fuel_type"].fillna("Electric" , inplace=True)
test_data["fuel_type"].fillna("Electric" , inplace=True)
mode_value_1 = train_data["clean_title"].mode()[0]
mode_value_2 = train_data["accident"].mode()[0]
mode_value_3 = train_data["clean_title"].mode()[0]
mode_value_4 = test_data["accident"].mode()[0]
train_data["clean_title"].fillna(mode_value_1 , inplace= True)
train_data["accident"].fillna(mode_value_1 , inplace= True)
test_data["clean_title"].fillna(mode_value_3 , inplace=True)
test_data["accident"].fillna(mode_value_4 , inplace=True)
train_data.drop(["id"] , axis=1 , inplace=True)
test_data.drop(["id"] , axis=1 , inplace=True)
train_data.drop(["engine"] , axis=1 , inplace=True)
test_data.drop(["engine"] , axis=1 , inplace=True)

C:\Users\TRITON\AppData\Local\Temp\ipykernel_2852\1843584948.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data["fuel_type"].fillna("Electric" , inplace=True)
C:\Users\TRITON\AppData\Local\Temp\ipykernel_2852\1843584948.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

In [30]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

objects = ['brand', 'model', 'fuel_type', 
            'transmission', 'ext_col', 'int_col',
            'accident', 'clean_title', 'brand_category']
for col in objects:
    le = LabelEncoder()
    combined = pd.concat([train_data[col], test_data[col]], axis=0).astype(str)
    le.fit(combined)
    train_data[col] = le.transform(train_data[col].astype(str))
    test_data[col] = le.transform(test_data[col].astype(str))

In [31]:
x_train = train_data.drop(["price"] , axis=1)
y_train = train_data["price"]

In [32]:
from sklearn.feature_selection import mutual_info_classif

mi_scores = mutual_info_classif(x_train , y_train , random_state=42)
mi_scores = pd.DataFrame({
    "Features" : x_train.columns,
    "mi_scores" : mi_scores
}).sort_values("mi_scores",ascending=False)

In [33]:
print(mi_scores)

          Features  mi_scores
3           milage   0.682026
2       model_year   0.553116
5     transmission   0.440251
1            model   0.287033
6          ext_col   0.152254
0            brand   0.141865
7          int_col   0.124737
8         accident   0.064431
10  brand_category   0.052003
4        fuel_type   0.032746
9      clean_title   0.000000
